## Input : Données Brut récupérées à partir du site Kaggle 
Source de données : https://www.kaggle.com/sobhanmoosavi/us-accidents



*© 2020, Ilham Noumir  (mailto:ilham.noumir@gmail.com)<br>



In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import datetime


In [2]:
df= pd.read_csv('US_Accidents.csv')

In [3]:
df.head(2)

,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,MapQuest,201.0,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,MapQuest,201.0,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,...,False,False,False,False,False,False,Night,Night,Night,Day


In [4]:
df.shape

(2974335, 49)

## Renomer certains colones

In [5]:
df= df.rename(columns={'Precipitation(in)':'Precipitation', 'Wind_Speed(mph)':'Wind_Speed' , 'Visibility(mi)':'Visibility', 
                      'Temperature(F)': 'Temperature' , 'Pressure(in)': 'Pressure' , 'Humidity(%)':'Humidity' , 'Distance(mi)':'Distance'})

## Chercher les enregistrements dupliqués 

In [6]:
df.duplicated().sum()

0

## Chercher les valeurs nulls

In [7]:
df.isnull().sum()

ID                             0
Source                         0
TMC                       728071
Severity                       0
Start_Time                     0
End_Time                       0
Start_Lat                      0
Start_Lng                      0
End_Lat                  2246264
End_Lng                  2246264
Distance                       0
Description                    1
Number                   1917605
Street                         0
Side                           0
City                          83
County                         0
State                          0
Zipcode                      880
Country                        0
Timezone                    3163
Airport_Code                5691
Weather_Timestamp          36705
Temperature                56063
Wind_Chill(F)            1852623
Humidity                   59173
Pressure                   48142
Visibility                 65691
Wind_Direction             45101
Wind_Speed                440840
Precipitat

## Nettoyage des données 

In [10]:
#########Fonction pour calculer le pourcentage des données manquantes par colone 

def missing_values_table(df):
        # Total des valeurs manquantes
        mis_val = df.isnull().sum()
        
        # pourcentage des valeurs manquants
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # construire une nouvelle dataframe avec les résultats
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Renomer les colones 
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Trier par odred décroissant
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Afficher quelques informations utiles 
        print ("ta base de donnée principale a " + str(df.shape[1]) + " colones.\n"      
            "Il y a " + str(mis_val_table_ren_columns.shape[0]) +
              " Colones qui ont des valeurs manquentes.")
        
        # Retourner la nouvelle dataframe avec le pourcentage des valeurs manquants 
        return mis_val_table_ren_columns
###### Afficher la nouvelle dataframe 
data_missing = missing_values_table(df)
##### Reset Index 
data_missing.reset_index(inplace=True)
##### Renomer la cololne 
data_missing = data_missing.rename(columns={'index':'name_of_colone'})

data_missing


ta base de donnée principale a 49 colones.
Il y a 23 Colones qui ont des valeurs manquentes.


,name_of_colone,Missing Values,% of Total Values
0,End_Lat,2246264,75.5
1,End_Lng,2246264,75.5
2,Precipitation,1998358,67.2
3,Number,1917605,64.5
4,Wind_Chill(F),1852623,62.3
5,TMC,728071,24.5
6,Wind_Speed,440840,14.8
7,Weather_Condition,65932,2.2
8,Visibility,65691,2.2
9,Humidity,59173,2.0


In [11]:
#### Colone TMC: Remplacer le TMC manquantes en code général d'accidents qui est le 201  qui représente déja µ65 % des données actuelles
df['TMC'].fillna(value= 201 , inplace = True)

In [12]:
##### Selectionner les colones qui ont un pourcentage supérieur à 60  pour les supprimer 
data_delete= data_missing.loc[data_missing['% of Total Values']> 60]

#### Fonction pour supprimer les colones dans la dataframe 
def delete_nan(missing,data): 
    for row in missing.itertuples():
        name= row.name_of_colone
        data.drop([name], axis='columns', inplace=True)
        
#### Appel de la fonction de suppression 
delete_nan(data_delete, df)

##### Selectionner les colones qui ont un pourcentage inférieur à 60  pour les traiter

data_missing= data_missing.loc[data_missing['% of Total Values']< 60]

#### Fonction pour traiter les colones numériques 
def traitement_nan(missing,data): 
    for row in missing.itertuples():
        name= row.name_of_colone
        if data[name].dtype =='float64':
            moyenne = round(data[name].mean(),0)
            data[name].fillna(moyenne, inplace=True)


##### Appel de la fonction : 

traitement_nan(data_missing, df)

#### suppression des valeurs manqauntes pour traiter les colones numériques 
df = df.dropna()

In [14]:
data_missing = missing_values_table(df)
data_missing

ta base de donnée principale a 44 colones.
Il y a 0 Colones qui ont des valeurs manquentes.


,Missing Values,% of Total Values


## Reconversion des colones en datetime

In [11]:
##### Convertir la colone startime et endtime
df['Start_Time'] = pd.to_datetime(df['Start_Time'] , errors= 'coerce')
df['End_Time'] = pd.to_datetime(df['End_Time'] , errors= 'coerce')
df['Weather_Timestamp']=  pd.to_datetime(df['Weather_Timestamp'] , errors= 'coerce')

## Suppression des valeurs abérantes 

In [12]:
df['dure'] = df['End_Time'] - df['Start_Time']

In [13]:
df = df[df['End_Time'].dt.year != 2020]

In [14]:
df = df[(df['dure'] >= '0 days')& (df['dure']  < '2 days')]

In [15]:
del df['dure']

##  Output : fichier csv Clean 

In [21]:
###### Créer un nouveau fichier csv clean 
df.to_csv('data_Us.csv')